In [1]:
# es5.1 & kibana5.1起動
!cd docker && docker-compose up --build -d

Building elasticsearch
Step 1/6 : FROM elasticsearch:5.1
 ---> 3b9ab09f293c
Step 2/6 : RUN bin/elasticsearch-plugin install analysis-icu
 ---> Using cache
 ---> 08aba46dc793
Step 3/6 : RUN bin/elasticsearch-plugin install analysis-kuromoji
 ---> Using cache
 ---> 72c11d566500
Step 4/6 : RUN bin/elasticsearch-plugin install analysis-phonetic
 ---> Using cache
 ---> 0cfb50a74429
Step 5/6 : EXPOSE 9200
 ---> Using cache
 ---> 62d7ee26a99c
Step 6/6 : CMD elasticsearch -Etransport.host=0.0.0.0
 ---> Using cache
 ---> cab13a608c59
Successfully built cab13a608c59
Building kibana
Step 1/1 : FROM kibana:5.1
 ---> 244bc3968631
Successfully built 244bc3968631
docker_elasticsearch_1 is up-to-date
docker_kibana_1 is up-to-date


In [2]:
# -*- coding: utf-8 -*-
%load_ext autoreload
%reload_ext autoreload
%autoreload 2

import sys
import json
import pprint
sys.path.append("lib/")
import pandas as pd
import numpy as np
from elasticsearch import Elasticsearch
import graphviz as gv

INDEX='my-index'
TYPE_NAME='my-type'

es = Elasticsearch(
    host='localhost',
    port=9200,
    use_ssl=False
)
#es = Elasticsearch()

In [3]:
#2.3.0にあったchar_filters, filters が廃止
def analyzer(index, analyzer, char_filters, tokenizer, filters, text, explain=False):
    analyzer = es.indices.analyze(
        index=index,
        text=text,
        char_filter=char_filters,
        tokenizer=tokenizer,
        filter=filters,
        explain=explain
    )
    return analyzer

Analyzer
==


Tokenizer
-------------

Tokenizers | Elasticsearch Reference [5.1] | Elastic

https://www.elastic.co/guide/en/elasticsearch/reference/5.1/analysis-tokenizers.html

について学ぶ

種類
-------------

* Word Oriented Tokenizers
* Partial Word Tokenizersedit
* Structured Text Tokenizers

がある。

### Word Oriented Tokenizers

* [ ]standard
* [ ]letter
* [ ]lowercase
* [ ]whitespace
* [ ]uax_url_email
* [ ]classic
* [ ]thai

### Partial Word Tokenizersedit

* [ ]ngram
* [ ]edge_ngram

### Structured Text Tokenizers

- [x] keyword
- [x] pattern
- [x] path_hierarchy



Partial Word Tokenizersedit: ngram
-----------
https://www.elastic.co/guide/en/elasticsearch/reference/current/analysis-ngram-tokenizer.html
n文字で切り取り

In [4]:
char_filters = ""
tokenizer = "ngram"
filters = ""

text="吾輩は猫である。名前はまだない。"
print(json.dumps([ i["token"] for i in analyzer(INDEX, analyzer, char_filters, tokenizer, filters, text, explain=False)["tokens"] ], indent=4, ensure_ascii=False))

[
    "吾", 
    "吾輩", 
    "輩", 
    "輩は", 
    "は", 
    "は猫", 
    "猫", 
    "猫で", 
    "で", 
    "であ", 
    "あ", 
    "ある", 
    "る", 
    "る。", 
    "。", 
    "。名", 
    "名", 
    "名前", 
    "前", 
    "前は", 
    "は", 
    "はま", 
    "ま", 
    "まだ", 
    "だ", 
    "だな", 
    "な", 
    "ない", 
    "い", 
    "い。", 
    "。"
]


In [5]:
body="""
{
  "settings": {
    "analysis": {
      "tokenizer": {
        "ngram_tokenizer": {
          "type": "ngram",
          "min_gram": 3,
          "max_gram": 3
        }
      }
    }
  }
}
"""

es.indices.close(index=INDEX)
es.indices.put_settings(index=INDEX, body=body)
es.indices.open(index=INDEX)

{u'acknowledged': True}

In [6]:
char_filters = ""
tokenizer = "ngram_tokenizer"
filters = ""

text="吾輩は猫である。名前はまだない。"
print(json.dumps([ i["token"] for i in analyzer(INDEX, analyzer, char_filters, tokenizer, filters, text, explain=False)["tokens"] ], indent=4, ensure_ascii=False))

[
    "吾輩は", 
    "輩は猫", 
    "は猫で", 
    "猫であ", 
    "である", 
    "ある。", 
    "る。名", 
    "。名前", 
    "名前は", 
    "前はま", 
    "はまだ", 
    "まだな", 
    "だない", 
    "ない。"
]


Partial Word Tokenizersedit: edge_ngram
-----------

https://www.elastic.co/guide/en/elasticsearch/reference/current/analysis-edgengram-tokenizer.html



In [7]:
text="吾輩は猫である。名前はまだない。"

char_filters=""
tokenizer = "edge_ngram"
filters=""


print(json.dumps([ i["token"] for i in analyzer(INDEX, analyzer, char_filters, tokenizer, filters, text, explain=False)["tokens"] ], indent=4, ensure_ascii=False))

[
    "吾", 
    "吾輩"
]


### edge_ngram tokenizerその２

30文字まで 区切ってみる


In [8]:
body="""
{
  "settings": {
    "analysis": {
      "tokenizer": {
        "engram_tokenizer": {
          "type": "edge_ngram",
          "min_gram": 2,
          "max_gram": 10
        }
      }
    }
  }
}
"""

es.indices.close(index=INDEX)
es.indices.put_settings(index=INDEX, body=body)
es.indices.open(index=INDEX)


{u'acknowledged': True}

In [9]:
text="吾輩は猫である。名前はまだない。"

char_filters=[]
tokenizer = "engram_tokenizer"
filters=[]

print(json.dumps([ i["token"] for i in analyzer(INDEX, analyzer, char_filters, tokenizer, filters, text, explain=False)["tokens"] ], indent=4, ensure_ascii=False))

[
    "吾輩", 
    "吾輩は", 
    "吾輩は猫", 
    "吾輩は猫で", 
    "吾輩は猫であ", 
    "吾輩は猫である", 
    "吾輩は猫である。", 
    "吾輩は猫である。名", 
    "吾輩は猫である。名前"
]
